## Studying the binary system of Neutron Star

### TOV - before using Real Data

### The theory

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
#####

In [ ]:
# Constants (SI)                 
K  = 1e-2         
gamma = 2.0

# Constants 
G = 1           # Work in geometrized units:  G = c = 1
c = 1
M_sun = 1.98847e30
km = 1e3         

# Polytropic EOS parameters 
K =  1e-1
gamma = 2.0

In [ ]:
####

In [ ]:
# TOV model of Presure and density
def eos_pressure(rho):
    return K * rho**gamma

def eos_rho_from_P(P):
    return (P / K) ** (1.0 / gamma)

def tov_rhs(r, y):
    P, m = y
    if P <= 0:
        return [0, 0]
    rho = eos_rho_from_P(P)
    dPdr = - (rho * c**2 + P) * (G * m + 4 * np.pi * G * P * r**3 / c**2) / (r * (r - 2 * G * m / c**2))
    dmdr = 4 * np.pi * r**2 * rho
    return [dPdr, dmdr]

def integrate_star(rho_c):
    P0 = eos_pressure(rho_c)
    y0 = [P0, 0.0]
    r0 = 1.0                    # start at 1 m to avoid singularity
    r_max = 2e6                   # integrate up to 2000 km
    
    sol = solve_ivp(tov_rhs, (r0, r_max), y0, max_step= 100.0, dense_output= True, events= lambda r, y: y[0])
    R = sol.t_events[0][0] if sol.t_events[0].size>0 else sol.t[-1]
    M = sol.y[1][-1]
    return R, M

In [ ]:
####

In [ ]:
central_rhos = np.logspace(17, 18.5, 20)       # kg m^-3
R_list= []; M_list= []

for rho_c in central_rhos:
    R, M = integrate_star(rho_c)
    R_km = R / km
    M_sun = M / Msun
    R_list.append(R_km)
    M_list.append(M_sun)

In [ ]:
####

In [ ]:
plt.figure(figsize= (7,5))
plt.plot(M_list, R_list, marker= 'o')
plt.xlabel("Radius (km)")
plt.ylabel("Mass (solar mass)")
plt.title("Mass–Radius curve (polytropic γ=2)")
plt.grid(True)
plt.show()